# BinaryBrain による一般的な多層パーセプトロン

BinaryBrainを使って、一般的な多層パーセプトロンを構成してMNISTを試します。
これはBinaryBrainのFPGA化対象外のネットなので計算しておしまいです。

In [1]:
import os
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

データセットの準備には PyTorch の torchvision を流用します

In [2]:
# configuration
net_name        = 'MnistMlp'
data_path       = os.path.join('./data/', net_name)
epochs          = 4
mini_batch_size = 64

# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

ネットワークを定義します。
最後の層は Loss関数側で LossSoftmaxCrossEntropy を使うので活性化層を付けません。

In [3]:
# ネット定義
net = bb.Sequential([
            bb.DenseAffine([1024]),
            bb.ReLU(),
            bb.DenseAffine([512]),
            bb.ReLU(),
            bb.DenseAffine([10]),
        ])
net.set_input_shape([28, 28])

[10]

損失関数/評価関数/最適化器を作ります。
最適化器にはネットワークの重みパラメータと勾配を紐づけます

In [4]:
# 損失関数/評価関数/最適化
loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

実際の学習を行います。
エポックごとにテストデータセットで評価も行います。

In [5]:
for epoch in range(epochs):
    # 学習
    for images, labels in loader_train:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
        y_buf = net.forward(x_buf, train=True)
        dy_buf = loss.calculate(y_buf, t_buf)
        net.backward(dy_buf)
        optimizer.update()

    # 評価
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
        y_buf = net.forward(x_buf, train=False)
        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))
    
    bb.save_networks(data_path, net)

epoch[0] : loss=0.164124 accuracy=0.975200
epoch[1] : loss=0.119439 accuracy=0.973250
epoch[2] : loss=0.097397 accuracy=0.972300
epoch[3] : loss=0.083052 accuracy=0.974175
